# Homework-week3
## 1. 构建Seq2seq模型中的Encoder层，建议采用面向对象的编程形式。
部分代码参考, 尝试不同的单元结构，考虑加上预训练词向量
```python
class Encoder():
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        # code
    def call(self, x, hidden):
        # code
        pass

    def initialize_hidden_state(self):
        pass
```

## 2. 构建Seq2seq模型中的Dncoder层，建议采用面向对象的编程形式。
部分代码参考, 尝试不同的单元结构，考虑加上预训练词向量
```python
class Decoder():
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Decoder, self).__init__()
        # code
    def call(self, x, hidden, enc_output, context_vector):
        # code
        pass
```

## 3. 构建Seq2seq模型中的Attention层，建议采用面向对象的编程形式。<u>这部分这次作业选做</u>。
部分代码参考
```python
class BahdanauAttention():
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        # code

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        # code

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        # code

        # attention_weights shape == (batch_size, max_length, 1)
        # code

        # context_vector shape after sum == (batch_size, hidden_size)
        # code

        return 
```





In [1]:
import sys
sys.path.append("/home/abner/PycharmProjects/NLP_Pro_01")

In [2]:
import tensorflow as tf

In [3]:
import utils.config

In [4]:
from gensim.models import word2vec

In [5]:
w2v_model = word2vec.Word2Vec.load(utils.config.w2v_model_path)

In [10]:
model_vetor_size = len(w2v_model.wv.vocab)

In [11]:
model_embedding_dim = w2v_model.vector_size

In [17]:
tf.config.experimental.list_logical_devices("GPU")

[LogicalDevice(name='/job:localhost/replica:0/task:0/device:GPU:0', device_type='GPU')]

参数说明：
vocab_size 词表数量
embedding_dim 词向量维度
enc_units 单元数（encoder层中包含的单元，如RNN、LSTM、GRU
batch_sz 分片大小

In [ ]:
class Encoder():
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_size = batch_sz
        self.enc_units = enc_units
        self.embedding_dim = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        gpus = tf.config.experimental.list_logical_devices("GPU")
        if gpus:
            self.gru = tf.keras.layers.CuDNNGRU(self.enc_units,
                                                return_sequences=True,
                                                return_state=True,
                                                recurrent_initializer='glorot_uniform')
        else:
            self.gru = tf.keras.layers.GRU(self.enc_units,
                                           return_sequences=True,
                                           return_state=True,
                                           recurrent_initializer='glorot_uniform')
    def call(self, x, hidden):
        x = self.embedding_dim(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
class Decoder():
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            self.gru = tf.keras.layers.CuDNNGRU(self.enc_units,
                                                return_sequences=True,
                                                return_state=True,
                                                recurrent_initializer='glorot_uniform')
        else:
            self.gru = tf.keras.layers.GRU(self.enc_units,
                                           return_sequences=True,
                                           return_state=True,
                                           recurrent_initializer='glorot_uniform')
    def call(self, x, hidden, enc_output, context_vector):
        output, state = self.gru(enc_output, initial_state=hidden)
        return output, state

老师例子

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            self.gru = tf.keras.layers.CuDNNGRU(self.enc_units,
                                                return_sequences=True,
                                                return_state=True,
                                                recurrent_initializer='glorot_uniform')
        else:
            self.gru = tf.keras.layers.GRU(self.enc_units,
                                           return_sequences=True,
                                           return_state=True,
                                           recurrent_initializer='glorot_uniform')
        # self.bigru = tf.keras.layers.Bidirectional(self.gru, merge_mode='concat')

    def call(self, x, hidden):
        x = self.embedding(x)
        # hidden = tf.split(hidden, num_or_size_splits=2, axis=1)
        # output, forward_state, backward_state = self.bigru(x, initial_state=hidden)
        # state = tf.concat([forward_state, backward_state], axis=1)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))